# Goal:

- provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

- Used as a Python sanity check because I'm not that familiar with R.

In [2]:
import numpy as np
np.random.seed(42)
import pandas as pd
import tqdm
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from numba import njit

np.seterr(all='raise')

{'divide': 'raise', 'over': 'raise', 'under': 'raise', 'invalid': 'raise'}

# Load and Process the Data

In [3]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../cscore_data/X_train_79999.csv"
    y_file = "../cscore_data/y_train_79999.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [57]:
NUM_SAMPLES = -1
COMPARE_TO_R_REF = False
lr = 0.001
mu = 0.5
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)

# Same shape as Marcelo's reference code
betas = np.zeros((train_x.shape[1], ))

Using subsampled data to compare Python-C++
Reading in ../cscore_data/X_train_79999.csv, ../cscore_data/y_train_79999.csv 
Train X shape is: (79998, 47)
Train y shape is: (79998, 1)


In [58]:
train_x[0, :5]

array([-0.34660179,  0.39375239,  0.50834598,  0.51504625,  0.49324492])

In [59]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas   # A vector of linear_predictions/logits z = train_x @ weights
    if dbg:
        print(f"Logits: {preds}")
    return np.expand_dims(sigmoid(preds), -1)   # Shape: (m, 1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)   # A vector of logistic predictions y_hat = sigmoid(z)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient   # Shape: (10, 1) == Rows correspond to num_features
    ## This function is used to update the values of betas: w_new = w_old + lr * gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))   # h: hypothesis, basically preds/y_hat
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    ## np.clip() function prevents computing the log of 0, by taking the minimum of 1e-15.
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]   # Shape: (1,) == scalar value

def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    for i in tqdm.trange(epochs):
    # for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)

        ## Assign updated weights into phi_prime
        phi_prime = theta - lr * np.squeeze(gradient)   # np.squeeze() removes single dimensions --> shape (10,)
        
        ## Nesterov acceleration process
        if i == 0:
            theta = phi_prime
        else:
            ## If current updated weight (phi_prime) < previous weight (phi), 
            ## The updated weight theta will be even smaller.
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime   # phi is then the weight of the previous epoch/update
        loss = cost(train_x, train_y, theta)
        nesterov_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return nesterov_loss, theta, phi


In [60]:
losses, theta, phi = nesterov(betas, 800, lr, mu, train_x, train_y)

100%|██████████| 800/800 [00:23<00:00, 33.99it/s]


In [61]:
losses

## zero initialized betas    : cost = 0.4087381183990924
## uniform initialized betas : cost = 0.40867096414830645
## randn initialized betas   : cost = 0.40850863604124726

[0.6928668405457935,
 0.6924470933574226,
 0.6919585365638609,
 0.6914364844403184,
 0.6908985878005592,
 0.6903536638473698,
 0.6898061143045032,
 0.6892581343929911,
 0.6887108168507253,
 0.6881647036020259,
 0.6876200613958829,
 0.687077019520237,
 0.6865356386050769,
 0.6859959449965628,
 0.6854579479302745,
 0.6849216481108946,
 0.6843870419985637,
 0.6838541239502979,
 0.6833228872898047,
 0.6827933248419283,
 0.6822654291996252,
 0.6817391928573098,
 0.6812146082774434,
 0.6806916679237656,
 0.680170364277862,
 0.6796506898474077,
 0.6791326371702473,
 0.6786161988163982,
 0.6781013673890132,
 0.6775881355248231,
 0.6770764958943203,
 0.6765664412018101,
 0.676057964185396,
 0.6755510576169327,
 0.6750457143019623,
 0.6745419270796374,
 0.6740396888226446,
 0.6735389924371202,
 0.6730398308625666,
 0.6725421970717657,
 0.6720460840706911,
 0.6715514848984209,
 0.6710583926270466,
 0.6705668003615827,
 0.6700767012398764,
 0.669588088432513,
 0.6691009551427239,
 0.66861529460629

In [62]:
theta

array([ 0.01213015,  0.02275907,  0.03178326, -0.06977391, -0.04087975,
       -0.07481257, -0.04178215, -0.12001809, -0.08493469, -0.06014307,
       -0.05278674, -0.0494054 ,  0.00513388,  0.06871688, -0.00098964,
        0.02472165,  0.02272765, -0.04809864, -0.04711155, -0.0539262 ,
       -0.05539888, -0.04640654, -0.04835133, -0.05418346, -0.0480884 ,
       -0.09540715, -0.01873639, -0.20528923, -0.08213223, -0.04964658,
       -0.07619056, -0.27572788, -0.03087855, -0.0502227 , -0.06626205,
       -0.04508224, -0.04253107, -0.05557824, -0.11101017, -0.03766481,
        0.00189356,  0.00049217, -0.05207346, -0.03199054, -0.10932508,
       -0.1503705 , -0.14186943])

In [70]:
pred = fwd(train_x, theta, dbg=False)

## Decision (Threshold = 0.5)
train_y_hat = (pred >= 0.4).astype(int)

In [71]:
train_y_hat.shape

(79998, 1)

In [72]:
train_y.shape

(79998, 1)

In [73]:
confusion_matrix(train_y, train_y_hat)

array([[45509, 20109],
       [ 2053, 12327]], dtype=int64)

In [67]:
roc_auc_score(train_y, pred)

0.8396195860467914

In [68]:
accuracy_score(train_y, train_y_hat)

0.8161954048851221

In [69]:
f1_score(train_y, train_y_hat)

0.5527436427789268